In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [3]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [ ]:
# Load all files
for n in range(1,nfiles+1):

    with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
      infiles = json.load(f)
    
    filename = '/myeosdir/vh-category/outfiles/'+year+'_'+str(n)+'.coffea'
    #filename = 'outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print(n,infiles.keys())
        #print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates'].scale(scale_lumi, 'dataset')
outsum['templates-vh-1'].scale(scale_lumi, 'dataset')
outsum['templates-vh-2'].scale(scale_lumi, 'dataset')
outsum['templates-vh-3'].scale(scale_lumi, 'dataset')

templates = outsum['templates'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal').integrate('systematic', 'nominal')
templates1 = outsum['templates-vh-1'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal')
templates2 = outsum['templates-vh-2'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal')
templates3 = outsum['templates-vh-3'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal')
del outsum

print('ok')

In [ ]:
# check intergrals
templates.sum('pt1','msd1','ddb1','DR')

In [ ]:
ptbins = [450, 500, 550, 600, 675, 800, 1200]
os.system('rm '+year+'/signalregion.root')
fout = uproot3.create(year+'/signalregion.root')
#for i,b in enumerate(ptbins[:-1]):
#    for p in pmap.keys():  
#        print(p)
#        for s in systematics:
#            h = templates.integrate('region', 'signal').integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
#            fout[p+"_pass_pt"+str(i+1)+"_"+s] = hist.export1d(h)
#            h = templates.integrate('region', 'signal').integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
#            fout[p+"_fail_pt"+str(i+1)+"_"+s] = hist.export1d(h)

fout.close()

In [ ]:
#os.system('rm '+year+'/muonCR.root')
#fout = uproot3.create(year+'/muonCR.root')
#for p in pmap.keys():  
#    print(p)
#    for s in systematics:
#        # muonkin
#        h = templates.integrate('region', 'muoncontrol').integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
#        fout[p+"_pass_"+s] = hist.export1d(h)
#        h = templates.integrate('region', 'muoncontrol').integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
#        fout[p+"_fail_"+s] = hist.export1d(h)